<a href="https://colab.research.google.com/github/blacker32/llm-guide/blob/main/rss_news_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=73c8dd17963f70727e5c639bcd8d0bd05ca9c215673c351b9ad060b60c0f6aaf
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [3]:
import feedparser
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import MarianMTModel, MarianTokenizer
from time import sleep
from datetime import datetime

# Çeviri modeli ve tokenizer'ı başlat
model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Çeviri fonksiyonu
def translate_text(src_text):
    try:
        inputs = tokenizer(src_text, return_tensors="pt", padding=True)
        translated = model.generate(**inputs)
        return [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    except Exception as e:
        print(f"Çeviri hatası: {e}")
        return src_text  # Çeviri başarısız olursa orijinal metni döndür

# RSS akışını getir
def fetch_feed(feed_url):
    try:
        response = requests.get(feed_url)
        response.raise_for_status()  # Başarısız yanıtlar için hata fırlatır
        return parse_feed(response.content)
    except Exception as e:
        print(f"{feed_url} akışı alınırken hata oluştu: {e}")
        return []

# Akış içeriğini ayıklama
def parse_feed(feed_content):
    feed = feedparser.parse(feed_content)
    articles = []
    for entry in feed.entries:
        article = {
            'title': entry.title,
            'link': entry.link,  # Makale URL'sini al
            'published': entry.published,
            'content': entry.get('content', [{'value': ''}])[0].get('value', '') or entry.get('summary', ''),
            'category': entry.get('category', 'N/A'),
            'tags': [tag.term for tag in entry.tags] if hasattr(entry, 'tags') else [],
            'source': entry.get('source', 'N/A'),
            'media_url': entry.get('media_content', [{}])[0].get('url', None)  # Medya içeriği varsa ele al
        }
        articles.append(article)
    return articles

# Ana fonksiyon
def monitor_feeds(feeds, interval=60):
    while True:
        all_articles = []

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(fetch_feed, feed) for feed in feeds]
            for future in as_completed(futures):
                try:
                    articles = future.result()
                    all_articles.extend(articles)
                except Exception as e:
                    print(f"Bir hata oluştu: {e}")

        # Başlıkları ve içeriği çevir ve sonuçları yazdır
        for article in all_articles:
            # Başlık ve içeriği çevir
            translated_title = translate_text([article['title']])[0]
            translated_content = translate_text([article['content']])[0]

            # Çevirilmiş içeriği yazdır
            print(f"\nBaşlık (TR): {translated_title}")
            print(f"Yayın Tarihi: {article['published']}")
            print(f"Açıklama/Tam İçerik (TR): {translated_content}")
            print(f"Kategori: {article['category']}")
            print(f"Etiketler: {', '.join(article['tags']) if article['tags'] else 'Yok'}")
            print(f"Kaynak: {article['source']}")
            print(f"Makale URL: {article['link']}")  # Makale URL'sini yazdır
            print(f"Multimedya Bağlantısı: {article['media_url'] if article['media_url'] else 'Yok'}\n")

        print(f"\nGüncelleme tamamlandı: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        sleep(interval)

# İzlenecek RSS akışları listesi
rss_feeds = [
    "https://rss.nytimes.com/services/xml/rss/nyt/World.xml",
    "https://feeds.bbci.co.uk/news/world/rss.xml",
    "https://www.aljazeera.com/xml/rss/all.xml"
]

# Akışları izlemeye başla
monitor_feeds(rss_feeds, interval=300)  # Her 5 dakikada bir kontrol et


tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/833k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/470M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]


Başlık (TR): Botswana cumhurbaşkanı, partisinin 58 yıllık yönetimine son vererek seçimi kabul etti
Yayın Tarihi: Fri, 01 Nov 2024 10:41:15 +0000
Açıklama/Tam İçerik (TR): Botswana Devlet Başkanı Mokgweetsi Masisi seçim yenilgisini kabul ederek, parti&#039;s 58 yıllık kuralını sona erdirdi.
Kategori: Show Types
Etiketler: Show Types
Kaynak: N/A
Makale URL: https://www.aljazeera.com/program/newsfeed/2024/11/1/botswana-president-concedes-election-ending-his-partys-58-year-rule?traffic_source=rss
Multimedya Bağlantısı: Yok


Başlık (TR): Pakistan çocuk felci aşısı sürücüsüne yapılan saldırıda yedi çocuktan beşi öldü
Yayın Tarihi: Fri, 01 Nov 2024 10:30:44 +0000
Açıklama/Tam İçerik (TR): Başbakan Şehbaz Şerif, Balochistan'ın güneybatı eyaletindeki bir kız ve okul yakınındaki saldırıyı kınadı.
Kategori: News
Etiketler: News
Kaynak: N/A
Makale URL: https://www.aljazeera.com/news/2024/11/1/five-children-among-seven-killed-in-attack-on-pakistan-polio-vaccine-drive?traffic_source=rss
Multimedya

KeyboardInterrupt: 